In [1]:
import simpy
import pandas as pd

import random

In [2]:
class Printer:

    def __init__( self , env, name , speed_pps , capacity ):

        self.env = env
        self.name = name
        self.speed_pps = speed_pps

        self.tank = simpy.Container( env , init=capacity , capacity=capacity  )
        self.resource = simpy.Resource( env , capacity=1 )

        self.dfP = pd.DataFrame({
            "name" : [name]
            , "speed": [speed_pps]
            , "tank": [ capacity ]
        })

        self.dfE = pd.DataFrame()

    def print_pages( self , pages, person_name ):
        if self.tank.level > pages:
            yield self.tank.get( pages )
            print_time = pages * self.speed_pps
            yield self.env.timeout(1)
            
            print( f" { self.env.now }: { self.name } is printing { pages } pages in { print_time } secs " )
            df = pd.DataFrame({
                "time": [ self.env.now ]
                , "printer": [ self.name ]
                , "person": [ person_name ]
                , "event": [ "printing" ]
            })

            self.dfE = pd.concat( [self.dfE , df] )
            
            yield self.env.timeout( print_time )
        else:
            print( f" { self.env.now }: printer { self.name } now wo capacity!!!! :( " )
            df = pd.DataFrame({
                "time": [ self.env.now ]
                , "printer": [ self.name ]
                , "person": [ person_name ]
                , "event": [ "no resource" ]
            })

            self.dfE = pd.concat( [self.dfE , df] )
            
        

In [3]:
class Person:
    def __init__( self , env , name , printer , num_pages ):
        self.env = env
        self.name = name
        self.printer = printer
        self.num_pages = num_pages
        env.process( self.in_line() )

        self.dfP = pd.DataFrame({
            "name": [name]
            , "printer": [ printer.name ]
            , "num_pages": [ num_pages ]
        })

        self.dfE = pd.DataFrame()

    def in_line( self ):
        print( f" { self.env.now }: { self.name } is waiting for { self.printer.name } " )
        df = pd.DataFrame({
            "time": [ self.env.now ]
            , "printer": [ self.printer.name ]
            , "person": [ self.name ]
            , "event": [ "waiting in line" ]
        })
        self.dfE = pd.concat( [self.dfE , df] )
        with self.printer.resource.request() as request:
            yield request
            print( f" { self.env.now }: { self.name } is ready to start using { self.printer.name } " )
            yield self.env.process( self.printer.print_pages( self.num_pages , self.name ) )
            print( f" { self.env.now }: { self.name } was attended by { self.printer.name } w all pages printed " )
            df = pd.DataFrame({
                "time": [ self.env.now ]
                , "printer": [ self.printer.name ]
                , "person": [ self.name ]
                , "event": [ "finished" ]
            })
            self.dfE = pd.concat( [self.dfE , df] )
            yield self.env.timeout( 1 )
            

In [4]:
class Setup:
    def __init__( self , env , printers , num_persons ):
        self.env = env
        self.people = []
        self.num_persons = num_persons
        self.printers = [ Printer(
            env
            , f"Printer { i }"
            , random.randint( 2 , 7 )
            , random.randint( 1500 , 2200 )
        ) for i in range(printers) ]

        env.process( self.arriving() )

    def arriving(self):
        for i in range( self.num_persons ):
            self.people.append(Person(
                env
                , f"Person {i}"
                , random.choice( self.printers )
                , random.randint( 5, 25 )
            ))
            yield self.env.timeout( random.randint( 1 , 300 ) ) ## Las personas llegan aleatoriamente entre 1 - 300 segundos

## Iniciamos la simulación

In [6]:
random.seed(111)

env = simpy.Environment()

setup = Setup( env , printers=2 , num_persons=200 )

env.run( until=30000 ) ## 8 horas 

 0: Person 0 is waiting for Printer 1 
 0: Person 0 is ready to start using Printer 1 
 1: Printer 1 is printing 18 pages in 90 secs 
 87: Person 1 is waiting for Printer 0 
 87: Person 1 is ready to start using Printer 0 
 88: Printer 0 is printing 18 pages in 54 secs 
 91: Person 0 was attended by Printer 1 w all pages printed 
 142: Person 1 was attended by Printer 0 w all pages printed 
 374: Person 2 is waiting for Printer 0 
 374: Person 2 is ready to start using Printer 0 
 375: Printer 0 is printing 10 pages in 30 secs 
 405: Person 2 was attended by Printer 0 w all pages printed 
 612: Person 3 is waiting for Printer 1 
 612: Person 3 is ready to start using Printer 1 
 613: Printer 1 is printing 17 pages in 85 secs 
 698: Person 3 was attended by Printer 1 w all pages printed 
 755: Person 4 is waiting for Printer 1 
 755: Person 4 is ready to start using Printer 1 
 756: Printer 1 is printing 12 pages in 60 secs 
 816: Person 4 was attended by Printer 1 w all pages printed 


## recolectamos el dataframe de Impresoras y empezamos a unir el dataframe de Eventos

In [8]:
df_events = pd.DataFrame()

In [9]:
df_printers = pd.DataFrame()

for p in setup.printers:
    df_printers = pd.concat( [ df_printers , p.dfP ] )
    df_events = pd.concat( [ df_events , p.dfE ] )

In [10]:
df_printers

,name,speed,tank
0,Printer 0,3,1823
0,Printer 1,5,1698


## recolectamos el dataframe de Personas y terminamos de unir el dataframe de Eventos

In [12]:
df_people = pd.DataFrame()

for p in setup.people:
    df_people = pd.concat( [ df_people , p.dfP ] )
    df_events = pd.concat( [ df_events , p.dfE ] )

In [13]:
df_people

,name,printer,num_pages
0,Person 0,Printer 1,18
0,Person 1,Printer 0,18
0,Person 2,Printer 0,10
0,Person 3,Printer 1,17
0,Person 4,Printer 1,12
...,...,...,...
0,Person 195,Printer 1,24
0,Person 196,Printer 0,5
0,Person 197,Printer 1,8
0,Person 198,Printer 0,11


In [14]:
df_events

,time,printer,person,event
0,88,Printer 0,Person 1,printing
0,375,Printer 0,Person 2,printing
0,1097,Printer 0,Person 6,printing
0,1547,Printer 0,Person 8,printing
0,1966,Printer 0,Person 11,printing
...,...,...,...,...
0,29206,Printer 1,Person 197,finished
0,29262,Printer 0,Person 198,waiting in line
0,29296,Printer 0,Person 198,finished
0,29512,Printer 1,Person 199,waiting in line


## Run Analysis!

Cuantas paginas imprimieron cada impresora

In [17]:
summPrinters =  df_people.groupby(
    ["printer"]
    , as_index=False
).agg({
    "num_pages": [ "sum" ]
    , "name": ["count"]
})

In [18]:
summPrinters

,printer,num_pages,name
,,sum,count
0,Printer 0,1536,99
1,Printer 1,1650,101


In [19]:
summPrinters.columns = summPrinters.columns.droplevel(1)

In [20]:
df2 = df_printers.merge(
    summPrinters
    , left_on="name"
    , right_on="printer"
    , how="inner"
)

In [21]:
df2["FreePages"] = df2["tank"] - df2["num_pages"]

- num_pages: Total de paginas que la impresora imprimio
- free pages: tank (capacidad maxima de impresión - num_pages) cuanto quedo libre para imprimir

In [23]:
df2

,name_x,speed,tank,printer,num_pages,name_y,FreePages
0,Printer 0,3,1823,Printer 0,1536,99,287
1,Printer 1,5,1698,Printer 1,1650,101,48


Como podemos observar Printer 0 tenia una capacidad maxima de 1823 paginas, atendio 1536 y aún le quedo libre 287 paginas

In [25]:
sample = df_events.loc[
    df_events["person"] == "Person 100"
    , :
].sort_values(  "time"   )

Aplicamos una función de ventada LEAD, la ventana la definimos por ```person``` para saber en que segundo sucedera el siguiente evento

En clases tomamos como ejemplo una sola persona ``` Person 100``` para leer bien el calculo y aplicarlo despues a nuestro dataset completo

In [27]:
sample['lead'] = sample.groupby('person')['time'].shift(-1)

In [28]:
sample["time_on_event"] = sample["lead"] - sample["time"]

In [29]:
sample

,time,printer,person,event,lead,time_on_event
0,15161,Printer 0,Person 100,waiting in line,15162.0,1.0
0,15162,Printer 0,Person 100,printing,15213.0,51.0
0,15213,Printer 0,Person 100,finished,NaN,NaN


La persona 100 llego a la cola al segundo 15161 y fue atendida inmediatamente al siguiente segundo ya que no habia nadie en cola. Uso la impresora por 51 segundos


Ahora corramos el calculo de LEAD para saber cuanto tiempo estuvo cada persona en cada evento


In [31]:
sample2 = df_events.sort_values(  ["person","time"]   )

In [32]:
sample2['lead'] = sample2.groupby('person')['time'].shift(-1)

In [33]:
sample2["time_on_event"] = sample2["lead"] - sample2["time"]

In [34]:
sample2

,time,printer,person,event,lead,time_on_event
0,0,Printer 1,Person 0,waiting in line,1.0,1.0
0,1,Printer 1,Person 0,printing,91.0,90.0
0,91,Printer 1,Person 0,finished,NaN,NaN
0,87,Printer 0,Person 1,waiting in line,88.0,1.0
0,88,Printer 0,Person 1,printing,142.0,54.0
...,...,...,...,...,...,...
0,14934,Printer 1,Person 98,printing,15054.0,120.0
0,15054,Printer 1,Person 98,finished,NaN,NaN
0,15043,Printer 1,Person 99,waiting in line,15056.0,13.0
0,15056,Printer 1,Person 99,printing,15081.0,25.0


Ahora si quisieramos saber cuantas personas  esperaron mas de 1 segundo para ser atendidos

In [36]:
person_waiting = sample2.loc[ 
    ( sample2['time_on_event'] > 1 )
    &
    ( sample2['event'] == "waiting in line" )
, : ]

person_waiting

,time,printer,person,event,lead,time_on_event
0,17433,Printer 1,Person 115,waiting in line,17449.0,16.0
0,17486,Printer 1,Person 116,waiting in line,17556.0,70.0
0,17627,Printer 1,Person 117,waiting in line,17653.0,26.0
0,17682,Printer 1,Person 118,waiting in line,17755.0,73.0
0,18900,Printer 0,Person 126,waiting in line,18968.0,68.0
0,20900,Printer 1,Person 138,waiting in line,20966.0,66.0
0,20943,Printer 1,Person 139,waiting in line,21078.0,135.0
0,21867,Printer 1,Person 147,waiting in line,21877.0,10.0
0,22990,Printer 0,Person 156,waiting in line,23035.0,45.0
0,23072,Printer 0,Person 157,waiting in line,23112.0,40.0


In [37]:
person_waiting.shape

(25, 6)

En total fueron 25 personas. ahora analizemos por impresora y cual fue el tiempo promedio de espera

In [39]:
person_waiting.groupby(
    "printer"
    , as_index=False
).agg({
    "time_on_event": [ "mean" ]
    , "person": [ "count" ]
})

,printer,time_on_event,person
,,mean,count
0,Printer 0,39.333333,9
1,Printer 1,55.125000,16


Para ```impresora 0``` las 9 personas solo esperaron 39 segundos en promedio, y para la ```impresora 1``` en promedio esperarón casi el minuto

En que hora de la simulación las impresoras dejaron de seguir atendiendo personas ?

In [41]:
timemax = df_events.groupby(
    "printer"
    , as_index=False
).agg({
    "time": [ "max" ] 
})

timemax.columns = ["printer","time"]

timemax["time"] = timemax["time"] / 60 / 60

In [42]:
timemax

,printer,time
0,Printer 0,8.137778
1,Printer 1,8.209167


 tiempo maximo de simulación fue de 30000 segundos, 8.33 horas. por lo que las impresoras atendieron a las 200 personas en un tiempo casi justo

 De las 8.33 horas cuantas horas estuvieron activas y cuanto tiempo estuvieron paradas ? 

In [44]:
printing = sample2.loc[ 
    df_events["event"] == "printing" 
    , :
].groupby(
    ["printer"]
    , as_index = False 
).agg({
    "time_on_event": ["sum"]
})

printing.columns = [ "printer" , "time_used" ]
printing["time_used"] = printing["time_used"] / 60 / 60 
printing["time_slept"] = 8.33 - printing["time_used"]

In [78]:
printing

,printer,time_used,time_slept
0,Printer 0,0.496667,7.833333
1,Printer 1,0.863889,7.466111


Como se puede observar las impresoras no fueron usadas tanto, pero por la manera en como se simulo la llegada de las personas si atendieron personas hasta el ultimo momento

En promedio Cuanto tiempo estuvieron paradas las impresoras esperando a q llegara la siguiente persona ?

In [89]:
sample3 = df_events.sort_values( ["printer","time"] )

In [91]:
sample3["lead"] = sample3.groupby("printer")['time'].shift(-1)

In [95]:
sample3["time_on_event"] = sample3["lead"] - sample3["time"] 

In [105]:
print_waiting = sample3.loc[ 
    sample3["event"] == "finished" 
, : ].groupby(
    ["printer"]
    , as_index=False
).agg({
    "time_on_event": ["mean","median"]
})

print_waiting.columns = ["printer","mean","median"]
print_waiting["mean"] = print_waiting["mean"] / 60
print_waiting["median"] = print_waiting["median"] / 60


In [107]:
print_waiting

,printer,mean,median
0,Printer 0,4.168537,3.466667
1,Printer 1,3.536333,2.658333


En promedio las impresoras estaban entre 2.6  - 4.1 minutos esperando a que llegara la siguiente persona